## Import libraries

In [1]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib.pyplot import axes, title
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn import metrics
import seaborn as sns

## Load the data

In [2]:
# Load the data set for the fourth IoT node
benign_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.benign.csv")
#g_scan_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.scan.csv")
#g_junk_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.junk.csv")
#g_udp_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.udp.csv")
#g_tcp_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.tcp.csv")
g_combo_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.combo.csv")
#m_scan_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.scan.csv")
#m_ack_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.ack.csv")
#m_syn_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.syn.csv")
#m_udp_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.udp.csv")
#m_p_udp_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.udpplain.csv")

In [3]:
print
print('Shape==>', 'Benign shape: ', benign_4.shape, '//','Bashlite combo: ', g_combo_4.shape)

Shape==> Benign shape:  (175240, 115) // Bashlite combo:  (58152, 115)


## Description

-- The following describes each of the features headers:

* Stream aggregation:
H: Stats summarizing the recent traffic from this packet's host (IP)
HH: Stats summarizing the recent traffic going from this packet's host (IP) to the packet's destination host.
HpHp: Stats summarizing the recent traffic going from this packet's host+port (IP) to the packet's destination host+port. Example 192.168.4.2:1242 -> 192.168.4.12:80
HH_jit: Stats summarizing the jitter of the traffic going from this packet's host (IP) to the packet's destination host.

* Time-frame (The decay factor Lambda used in the damped window):
How much recent history of the stream is capture in these statistics
L5, L3, L1, ...

* The statistics extracted from the packet stream:
weight: The weight of the stream (can be viewed as the number of items observed in recent history)
mean: ...
std: ...
radius: The root squared sum of the two streams' variances
magnitude: The root squared sum of the two streams' means
cov: an approximated covariance between two streams
pcc: an approximated covariance between two streams

## Prepare the data

In [4]:
benign=benign_4.sample(frac=0.25,replace=False)
g_combo=g_combo_4.sample(frac=0.25,replace=False)
#g_junk=g_junk_4.sample(frac=0.5,replace=False)
#g_scan=g_scan_4.sample(frac=0.5,replace=False)
#g_tcp=g_tcp_4.sample(frac=0.15,replace=False)
#g_udp=g_udp_4.sample(frac=0.15,replace=False)
#m_ack=m_ack_4.sample(frac=0.25,replace=False)
#m_scan=m_scan_4.sample(frac=0.15,replace=False)
#m_syn=m_syn_4.sample(frac=0.25,replace=False)
#m_udp=m_udp_4.sample(frac=0.1,replace=False)
#m_p_udp=m_p_udp_4.sample(frac=0.27,replace=False)


benign['type']='benign'
g_combo['type']='gafgyt_combo'
#g_junk['type']='gafgyt_junk'
#g_scan['type']='gafgyt_scan'
#g_tcp['type']='gafgyt_tcp'
#g_udp['type']='gafgyt_udp'
#m_udp['type']='mirai_udp'
#m_ack['type']='mirai_ack'
#m_scan['type']='mirai_scan'
#m_syn['type']='mirai_syn'
#m_p_udp['type']='mirai_udpplain'

In [5]:
#data=pd.concat([benign,m_udp,g_combo,g_junk,g_scan,g_tcp,g_udp,m_ack,m_scan,m_syn,m_p_udp],
#               axis=0, sort=False, ignore_index=True)

In [6]:
data=pd.concat([benign,g_combo],
               axis=0, sort=False, ignore_index=True)

In [7]:
#how many instances of each class
data.groupby('type')['type'].count()

type
benign          43810
gafgyt_combo    14538
Name: type, dtype: int64

In [8]:
data.shape

(58348, 116)

In [9]:
#shuffle rows of dataframe 
sampler=np.random.permutation(len(data))
data=data.take(sampler)
data.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,type
30069,1.003906,66.000000,9.950000e-07,1.035997,65.999831,1.015688e-03,1.429683,65.885333,1.662340,10.452283,...,-4.300000e-10,-1.050000e-06,18.488216,66.057505,0.828709,116.262918,217.397737,-0.245503,-0.020092,benign
3563,1.004649,66.000000,0.000000e+00,1.040758,66.000000,2.970000e-07,1.449353,66.025318,1.544761,9.867561,...,-7.938604e+00,-1.563288e-01,18.294705,70.153435,6.971186,99.914476,73.747515,-1.438903,-0.027714,benign
9406,1.002598,66.000000,2.000000e-06,1.028190,66.000151,1.358499e-02,1.422278,66.758823,67.718295,9.838075,...,-1.850000e-12,-7.200000e-08,19.149283,66.079987,0.976445,115.448327,238.893604,-0.361232,-0.023935,benign
46005,173.158759,74.008213,1.970465e-01,278.762557,74.017608,4.222831e-01,806.631568,74.033050,0.796567,7330.415908,...,0.000000e+00,0.000000e+00,1.677611,74.000000,0.000000,95.268043,0.000000,0.000000,0.000000,gafgyt_combo
36074,1.033480,66.000000,0.000000e+00,1.130304,66.000000,2.270000e-11,1.540639,66.000474,0.007630,8.924058,...,-5.680000e-11,-9.940000e-07,19.118733,66.034240,0.640086,115.691611,232.107760,-0.246350,-0.025262,benign


In [10]:
#dummy encode labels, store separately
labels_full=pd.get_dummies(data['type'], prefix='type')
labels_full.head()

,type_benign,type_gafgyt_combo
30069,1,0
3563,1,0
9406,1,0
46005,0,1
36074,1,0


In [11]:
#drop labels from training dataset
data=data.drop(columns='type')
data.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc
30069,1.003906,66.000000,9.950000e-07,1.035997,65.999831,1.015688e-03,1.429683,65.885333,1.662340,10.452283,...,8.124168e+01,-4.300000e-10,-1.050000e-06,18.488216,66.057505,0.828709,116.262918,217.397737,-0.245503,-0.020092
3563,1.004649,66.000000,0.000000e+00,1.040758,66.000000,2.970000e-07,1.449353,66.025318,1.544761,9.867561,...,7.355762e+01,-7.938604e+00,-1.563288e-01,18.294705,70.153435,6.971186,99.914476,73.747515,-1.438903,-0.027714
9406,1.002598,66.000000,2.000000e-06,1.028190,66.000151,1.358499e-02,1.422278,66.758823,67.718295,9.838075,...,1.447934e+02,-1.850000e-12,-7.200000e-08,19.149283,66.079987,0.976445,115.448327,238.893604,-0.361232,-0.023935
46005,173.158759,74.008213,1.970465e-01,278.762557,74.017608,4.222831e-01,806.631568,74.033050,0.796567,7330.415908,...,9.094947e-13,0.000000e+00,0.000000e+00,1.677611,74.000000,0.000000,95.268043,0.000000,0.000000,0.000000
36074,1.033480,66.000000,0.000000e+00,1.130304,66.000000,2.270000e-11,1.540639,66.000474,0.007630,8.924058,...,2.241835e+02,-5.680000e-11,-9.940000e-07,19.118733,66.034240,0.640086,115.691611,232.107760,-0.246350,-0.025262


In [12]:
#standardize numerical columns
def standardize(df,col):
    df[col]= (df[col]-df[col].mean())/df[col].std()

data_st=data.copy()
for i in (data_st.iloc[:,:-1].columns):
    standardize (data_st,i)

data_st.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc
30069,-0.582847,-0.171045,-0.095451,-0.581106,-0.173175,-0.097830,-0.576022,-0.176472,-0.107931,-0.567296,...,-0.147203,-0.035332,-0.018887,0.107372,-0.160534,-0.293008,-0.080878,-0.196836,-0.044407,-0.020092
3563,-0.582836,-0.171045,-0.095451,-0.581065,-0.173173,-0.097830,-0.575964,-0.174943,-0.107934,-0.567489,...,-0.147361,-0.037030,-1.127881,0.100243,-0.113353,-0.149582,-0.211880,-0.199550,-0.044627,-0.027714
9406,-0.582866,-0.171045,-0.095451,-0.581174,-0.173172,-0.097830,-0.576044,-0.166932,-0.105848,-0.567498,...,-0.145898,-0.035332,-0.018880,0.131726,-0.160275,-0.289558,-0.087406,-0.196430,-0.044428,-0.023935
46005,1.871299,-0.092685,-0.095445,1.833394,-0.091725,-0.097817,1.801526,-0.087483,-0.107958,1.844882,...,-0.148871,-0.035332,-0.018880,-0.511940,-0.069044,-0.312358,-0.249112,-0.200944,-0.044362,0.000000
36074,-0.582425,-0.171045,-0.095451,-0.580286,-0.173173,-0.097830,-0.575694,-0.175215,-0.107983,-0.567800,...,-0.144268,-0.035332,-0.018887,0.130601,-0.160802,-0.297412,-0.085456,-0.196558,-0.044407,-0.025262


# Different models

## DBSCAN - Density-Based Spatial Clustering of Applications with Noise

In [15]:
# Compute DBSCAN
db = DBSCAN(eps=4, min_samples=13000).fit(data_st) # variar eps e min_samples para encontrar n clusters desejado (use 13000)
db.labels_

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

for x in range(10, 0, -1):
    eps = 1/(11-x)
    samples = 130*x
    db = DBSCAN(eps=eps, min_samples=samples).fit(data_st)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    
    print('EPS: ', eps, 'Min samples: ', samples)
    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
    


In [17]:
classe1 = 0
classe2 = 0
outlier = 0
correct = 0
uncorrect = 0
outlier2 = 0
for i in range(len(data_st)):
    if(db.labels_[i] == 0):
        classe1+=1
    elif(db.labels_[i] == 1):
        classe2+=1
    elif(db.labels_[i] == -1):
        outlier+=1
    
    if((db.labels_[i] == 0 and labels_full.type_benign[data_st.index[i]] == 1) or
       (db.labels_[i] == 1 and labels_full.type_gafgyt_combo[data_st.index[i]] == 1)):
        correct+=1
    elif((db.labels_[i] == 0 and labels_full.type_benign[data_st.index[i]] == 0) or
       (db.labels_[i] == 1 and labels_full.type_gafgyt_combo[data_st.index[i]] == 0)):
        uncorrect+=1
    else:
        outlier2+=1
    
    
    
    #dbscan_label = db.labels_[i]
print('Benign length: ', len(benign), ' ', 'Gafgyt length: ', len(g_combo))
print('\nResults: ')
print('Grouped as: ', 'Benign: ', classe1, ' ', 'Malicious: ', classe2, ' ', 'Outlier: ', outlier)
print('Correct: ', correct,' ', 'Uncorrect: ', uncorrect,' ', 'Outliers: ', outlier2)

print('Accuracy: (%)', 'Benign data: ', classe1/len(benign), 'Malicious data: ', classe2/len(g_combo))
print('Complete data: ', correct/len(data))

Benign length:  43810   Gafgyt length:  14538

Results: 
Grouped as:  Benign:  40258   Malicious:  14391   Outlier:  3699
Correct:  54638   Uncorrect:  11   Outliers:  3699
Accuracy: (%) Benign data:  0.9189226204062999 Malicious data:  0.9898885678910442
Complete data:  0.9364159868375951


## Isolation Forests (IF)

In [30]:
clf = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto',
                      max_features=1.0, bootstrap=False, n_jobs=None, random_state=0,
                      verbose=0, warm_start=False).fit(data_st)
result = clf.predict(data_st) # -1 => outlier

In [31]:
normal = 0
malicious = 0
TP = 0; FP = 0; TN = 0; FN = 0

for i in range(len(data_st)):
    if(result[i] == 1):
        normal+=1
    elif(result[i] == -1):
        malicious+=1

    
    if(result[i] == 1 and labels_full.type_benign[data_st.index[i]] == 1):
        TP+=1
    elif(result[i] == 1 and labels_full.type_benign[data_st.index[i]] == 0):
        FN+=1
    elif(result[i] == -1 and labels_full.type_benign[data_st.index[i]] == 1):
        FP+=1
    elif(result[i] == -1 and labels_full.type_benign[data_st.index[i]] == 0):
        TN+=1
        

accuracy = (TP + TN)/(TP + TN + FP + FN)        
precision = TP/(TP + FP)
recall = TP/(TP+FN)
fscore = 2*precision*recall/(precision+recall)


print('Benign length: ', len(benign), ' ', 'Gafgyt length: ', len(g_combo))
print('\nResults: ')
print('Grouped as: ', 'Benign: ', normal, ' ', 'Malicious: ', malicious)

print('\nConfusion matrix: ', '\tTP:', TP, 'FP:', FP, '\n', '\t\t\t', 'FN:', FN, 'TN:', TN)
print('Accuracy (%): ', accuracy, '\nPrecision: ', precision, '\nRecall: ', recall, '\nF-score: ', fscore)


print('\nLegend:','\nTP: True Positive (Benign data classified as benign)', '\nFP: False Positive (Benign data classified as malicious)')
print('\nFN: False Negative (Malicious data classified as benign)', '\nTN: True Negative (Malicious data classified as malicious)')


Benign length:  43810   Gafgyt length:  14538

Results: 
Grouped as:  Benign:  56573   Malicious:  1775
Confusion matrix:  	TP: 42053 FP: 1757 
 			 FN: 14520 TN: 18
Accuracy (%):  0.7210358538424625 
Precision:  0.9598950011412919 
Recall:  0.7433404627649232 
F-score:  0.837851030552982

Legend: 
TP: True Positive (Benign data classified as benign) 
FP: False Positive (Benign data classified as malicious)

FN: False Negative (Malicious data classified as benign) 
TN: True Negative (Malicious data classified as malicious)


## Local Outlier Factor (LOF)

In [23]:
clf = LocalOutlierFactor(n_neighbors=20, algorithm='auto', leaf_size=30, 
                         metric='minkowski', p=2, metric_params=None, 
                         contamination='auto', novelty=False, n_jobs=None)
result = clf.fit_predict(data_st)
result

array([ 1,  1, -1, ...,  1,  1,  1])

In [24]:
normal = 0
malicious = 0
TP = 0; FP = 0; TN = 0; FN = 0

for i in range(len(data_st)):
    if(result[i] == 1):
        normal+=1
    elif(result[i] == -1):
        malicious+=1

    
    if(result[i] == 1 and labels_full.type_benign[data_st.index[i]] == 1):
        TP+=1
    elif(result[i] == 1 and labels_full.type_benign[data_st.index[i]] == 0):
        FN+=1
    elif(result[i] == -1 and labels_full.type_benign[data_st.index[i]] == 1):
        FP+=1
    elif(result[i] == -1 and labels_full.type_benign[data_st.index[i]] == 0):
        TN+=1
        

accuracy = (TP + TN)/(TP + TN + FP + FN)        
precision = TP/(TP + FP)
recall = TP/(TP+FN)
fscore = 2*precision*recall/(precision+recall)


print('Benign length: ', len(benign), ' ', 'Gafgyt length: ', len(g_combo))
print('\nResults: ')
print('Grouped as: ', 'Benign: ', normal, ' ', 'Malicious: ', malicious)

print('\nConfusion matrix: ', '\tTP:', TP, 'FP:', FP, '\n', '\t\t\t', 'FN:', FN, 'TN:', TN)
print('Accuracy (%): ', accuracy, '\nPrecision: ', precision, '\nRecall: ', recall, '\nF-score: ', fscore)


print('\nLegend:','\nTP: True Positive (Benign data classified as benign)', '\nFP: False Positive (Benign data classified as malicious)')
print('\nFN: False Negative (Malicious data classified as benign)', '\nTN: True Negative (Malicious data classified as malicious)')


Benign length:  43810   Gafgyt length:  14538

Results: 
Grouped as:  Benign:  54196   Malicious:  4152
Confusion matrix:  	TP: 39842 FP: 3968 
 			 FN: 14354 TN: 184
Accuracy (%):  0.6859875231370398 
Precision:  0.9094270714448756 
Recall:  0.7351465052771422 
F-score:  0.8130522621064017

Legend: 
TP: True Positive (Benign data classified as benign) 
FP: False Positive (Benign data classified as malicious)

FN: False Negative (Malicious data classified as benign) 
TN: True Negative (Malicious data classified as malicious)


## Elliptic Envelope (EE)

In [25]:
cov = EllipticEnvelope(store_precision=True, assume_centered=False, support_fraction=None,
                       contamination=0.1, random_state=None).fit(data_st) 
# maybe it is better to use Benign to fit the data and then use data_st to predict
# predict returns 1 for an inlier and -1 for an outlier
result = cov.predict(data_st)


C:\Users\gustavo\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Users\gustavo\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:166: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-2589.706287339770824 > -2592.140862478894178). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn("Determinant has increased; this should not happen: "
C:\Users\gustavo\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:166: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-2607.925281390894270 > -2612.026542038363914). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn("Determinant has increased; this sho

C:\Users\gustavo\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:166: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-2613.596608892560198 > -2613.958882861283200). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn("Determinant has increased; this should not happen: "
C:\Users\gustavo\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:166: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-2440.441793207758110 > -2440.660738512107400). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn("Determinant has increased; this should not happen: "
C:\Users\gustavo\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:166: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-2472.228984893595225 > -2473.38

In [26]:
normal = 0
malicious = 0
TP = 0; FP = 0; TN = 0; FN = 0

for i in range(len(data_st)):
    if(result[i] == 1):
        normal+=1
    elif(result[i] == -1):
        malicious+=1

    
    if(result[i] == 1 and labels_full.type_benign[data_st.index[i]] == 1):
        TP+=1
    elif(result[i] == 1 and labels_full.type_benign[data_st.index[i]] == 0):
        FN+=1
    elif(result[i] == -1 and labels_full.type_benign[data_st.index[i]] == 1):
        FP+=1
    elif(result[i] == -1 and labels_full.type_benign[data_st.index[i]] == 0):
        TN+=1
        

accuracy = (TP + TN)/(TP + TN + FP + FN)        
precision = TP/(TP + FP)
recall = TP/(TP+FN)
fscore = 2*precision*recall/(precision+recall)


print('Benign length: ', len(benign), ' ', 'Gafgyt length: ', len(g_combo))
print('\nResults: ')
print('Grouped as: ', 'Benign: ', normal, ' ', 'Malicious: ', malicious)

print('\nConfusion matrix: ', '\tTP:', TP, 'FP:', FP, '\n', '\t\t\t', 'FN:', FN, 'TN:', TN)
print('Accuracy (%): ', accuracy, '\nPrecision: ', precision, '\nRecall: ', recall, '\nF-score: ', fscore)


print('\nLegend:','\nTP: True Positive (Benign data classified as benign)', '\nFP: False Positive (Benign data classified as malicious)')
print('\nFN: False Negative (Malicious data classified as benign)', '\nTN: True Negative (Malicious data classified as malicious)')


Benign length:  43810   Gafgyt length:  14538

Results: 
Grouped as:  Benign:  52512   Malicious:  5836
Confusion matrix:  	TP: 37996 FP: 5814 
 			 FN: 14516 TN: 22
Accuracy (%):  0.651573318708439 
Precision:  0.8672905729285552 
Recall:  0.7235679463741621 
F-score:  0.7889371067876498

Legend: 
TP: True Positive (Benign data classified as benign) 
FP: False Positive (Benign data classified as malicious)

FN: False Negative (Malicious data classified as benign) 
TN: True Negative (Malicious data classified as malicious)


## One-Class Support Vector Machine (1CSVM)

In [27]:
clf = OneClassSVM(kernel='rbf', degree=3, gamma='auto',
                  coef0=0.0, tol=0.001, nu=0.35, shrinking=True,
                  cache_size=200, verbose=False, max_iter=- 1).fit(data_st)
result = clf.predict(data_st)

#clf.score_samples(X)

In [28]:
normal = 0
malicious = 0
TP = 0; FP = 0; TN = 0; FN = 0

for i in range(len(data_st)):
    if(result[i] == 1):
        normal+=1
    elif(result[i] == -1):
        malicious+=1

    
    if(result[i] == 1 and labels_full.type_benign[data_st.index[i]] == 1):
        TP+=1
    elif(result[i] == 1 and labels_full.type_benign[data_st.index[i]] == 0):
        FN+=1
    elif(result[i] == -1 and labels_full.type_benign[data_st.index[i]] == 1):
        FP+=1
    elif(result[i] == -1 and labels_full.type_benign[data_st.index[i]] == 0):
        TN+=1
        

accuracy = (TP + TN)/(TP + TN + FP + FN)        
precision = TP/(TP + FP)
recall = TP/(TP+FN)
fscore = 2*precision*recall/(precision+recall)


print('Benign length: ', len(benign), ' ', 'Gafgyt length: ', len(g_combo))
print('\nResults: ')
print('Grouped as: ', 'Benign: ', normal, ' ', 'Malicious: ', malicious)

print('\nConfusion matrix: ', '\tTP:', TP, 'FP:', FP, '\n', '\t\t\t', 'FN:', FN, 'TN:', TN)
print('Accuracy (%): ', accuracy, '\nPrecision: ', precision, '\nRecall: ', recall, '\nF-score: ', fscore)

print('\nLegend:','\nTP: True Positive (Benign data classified as benign)', '\nFP: False Positive (Benign data classified as malicious)')
print('\nFN: False Negative (Malicious data classified as benign)', '\nTN: True Negative (Malicious data classified as malicious)')


Benign length:  43810   Gafgyt length:  14538

Results: 
Grouped as:  Benign:  37929   Malicious:  20419
Confusion matrix:  	TP: 31421 FP: 12389 
 			 FN: 6508 TN: 8030
Accuracy (%):  0.6761328580242681 
Precision:  0.7172106824925816 
Recall:  0.8284162514171215 
F-score:  0.7688129289568015

Legend: 
TP: True Positive (Benign data classified as benign) 
FP: False Positive (Benign data classified as malicious)

FN: False Negative (Malicious data classified as benign) 
TN: True Negative (Malicious data classified as malicious)


## All 5 algorithms

In [ ]:
# Define "classifiers" to be used
algorithms = {
    "DBSCAN - Density-Based Spatial Clustering of Applications with Noise": DBSCAN(eps=4, min_samples=1300),
    "Isolation Forest": IsolationForest(random_state=0), 
    "Local Outlier Factor": LocalOutlierFactor(n_neighbors=20, contamination=0.1),
    "EllipticEnvelope": EllipticEnvelope(support_fraction=1., contamination=0.25),
    "OCSVM": OneClassSVM(nu=0.25, gamma=0.35)}

In [ ]:
for i, (alg_name, alg) in enumerate(algorithms.items()):
    alg.fit(data_st)
    ## show some results and evaluate them
    